<a href="https://colab.research.google.com/github/CloseChoice/FlatCurver/blob/dev/data_analysis/notebooks/Corona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading 


In [0]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

url = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/data/Coronavirus.history.v2.csv'
url_pop = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/data/einwohner_bundeslaender.csv'
url_rki_neuinfektionen = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/data/RKI_Neuinfektionen_pro_land_pro_tag.csv'
url_rki_todesfaelle = 'https://raw.githubusercontent.com/CloseChoice/FlatCurver/dev/data/RKI_Todesfaelle_pro_land_pro_tag.csv'

data = pd.read_csv(url)
df_ger = data.loc[data['parent'] == 'Deutschland'].copy()
df_infek = pd.read_csv(url_rki_neuinfektionen)
df_todesfaelle = pd.read_csv(url_rki_todesfaelle)

pop = pd.read_csv(url_pop, sep='\t')


df_ger = pd.merge(df_ger, pop.set_index('Bundesland'), left_on='label', right_index=True, how='left')
df_ger = df_ger.loc[df_ger['label'] != 'Repatriierte', :].copy()


In [0]:
df_todesfaelle.Altersgruppe.unique()

array(['A80+', 'A60-A79', 'A35-A59'], dtype=object)

# Mapbox

In [0]:

fig = px.scatter_mapbox(df_ger, lat="lat", lon="lon", size="confirmed", animation_frame='date',
                  color_continuous_scale=px.colors.cyclical.IceFire)
fig.update_layout(mapbox_style="open-street-map").update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                'mapbox': {'pitch': 10, 'zoom': 5}})
fig.show()

SyntaxError: ignored

In [0]:
fig = px.density_mapbox(df_ger, lat='lat', lon='lon', z='confirmed', radius=50, animation_frame='date', zoom=5, range_color=[0, max(df_ger['confirmed'])], height=800)
fig.update_layout(mapbox_style="open-street-map").update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                'mapbox': {'pitch': 10, 'zoom': 5}})
fig.show()

# Plots

In [0]:

fig = px.line(df_ger, x='date', y='confirmed', color='label')
fig.show()

In [0]:
fig = px.line(df_ger, x='date', y='confirmed', color='label', log_y=True)
fig.show()

# W.I.P.

,lon,lat,confirmed,recovered,deaths,Einwohner
date,,,,,,
2020-01-27,11.658165,48.768814,1,0,0,12843514.0
2020-01-28,11.658165,48.768814,4,0,0,12843514.0
2020-01-29,11.658165,48.768814,4,0,0,12843514.0
2020-01-30,11.658165,48.768814,5,0,0,12843514.0
2020-01-31,11.658165,48.768814,7,0,0,12843514.0


In [0]:
df_bundesweit = df_ger.groupby('date').sum().reset_index()

fig = px.line(df_bundesweit, x='date', y='confirmed', log_y=True)
fig.show()

## TODO
* Add functions for marker
* Add Date to Mapbox

In [0]:
df_bundesweit = df_ger.groupby('date').sum().reset_index()

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "mapbox"}, {"type": "xy"}]])
fig.add_trace(go.Densitymapbox(lat=df_ger['lat'], lon=df_ger['lon'], z=df_ger['confirmed']), row=1, col=1).update_layout(mapbox_style="open-street-map", mapbox_center_lon=11,
                                                                                                                         mapbox_center_lat=50).update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                'paper_bgcolor': 'rgba(0, 0, 0, 0)',
                'mapbox': {'pitch': 10, 'zoom': 5}})
fig.add_trace(go.Scatter(x=df_bundesweit['date'], y=df_bundesweit['confirmed'], name='Bundesweit'), row=1, col=2)
fig.update_layout(height=800, showlegend=False, title_text='Bundesweite Entwicklung',
    updatemenus=[
        dict(active=0,
            buttons=list([
                dict(
                    args=[{'yaxis': {'type': 'linear'}}],
                    label="Linear",
                    method="relayout"
                ),
                dict(
                    args=[{'yaxis': {'type': 'log'}}],
                    label="Log",
                    method="relayout"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x=0,
            y=1.075,
        ),
    ]
)
fig.show()